In [ ]:
import delfi.distribution as dd
import delfi.generator as dg
import delfi.inference as infer
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import scipy.stats as st
import time 

from lfimodels.balancednetwork.BalancedNetwork import BalancedNetwork
from lfimodels.balancednetwork.BalancedNetworkStats import BalancedNetworkStats

%matplotlib inline

### Define the objects we need for the simulation: 

model, prior, summarystats, a generator to combine it all

The parameter we use for the balanced network simulation is the clustering coef $R_{ee}$. For now we want this to be very close around 1. 

In [ ]:
n_params = 4

m = BalancedNetwork(dim=n_params, first_port=8010, 
                    verbose=False, n_servers=4, duration=3.)
p = dd.Uniform(lower=[0.01] * n_params, upper=[0.1] * n_params)
s = BalancedNetworkStats()
g = dg.Default(model=m, prior=p, summary=s)

### Make a test by running one sample = one simulation 

The generator returns the params used and the correspoding stats: the theta-x tuples used by the MDN. 

In [ ]:
m.start_server()

In [ ]:
# here we set the true params 
true_params = [[0.024, 0.045, 0.014, 0.057]]
# run forward model 
data = m.gen(true_params)
# get summary stats
stats_obs = s.calc(data[0])

In [ ]:
print(true_params, stats_obs)

In [ ]:
# set up inference
res = infer.Basic(g)
ntrain = 50
# run with N samples
out = res.run(ntrain, epochs=1000, minibatch=50)
m.stop_server()

In [ ]:
plt.plot(out['trn_iter'], out['trn_val'])

## Test the result: generate an observation and compare it to simulations from the posterior

In [ ]:
# evaluate the posterior at the observed data 
posterior = res.predict(stats_obs)

In [ ]:
# get the mean and std 
mean = posterior.xs[0].m
std = np.sqrt(posterior.xs[0].S)
print(mean)
print(std)

In [ ]:
plt.figure(figsize=(15, 10))
theta = np.linspace(0.01, 0.1, 100)
plot_idx = 1
for i in range(n_params): 
    for j in range(n_params): 
        if i==j: 
            plt.subplot(n_params, n_params, plot_idx)
            plot_idx += 1
            post_pdf = st.norm.pdf(x=theta, loc=mean[i], scale=std[i, i])
            plt.plot(theta, post_pdf)
            plt.axvline(x=true_params[0][i], color='C1')
        else: 
            sub_mean = [mean[i], mean[j]]
            sub_std = sub_std = [[std[i, i], std[i, j]], [std[j, i], std[j, j]]]
            if np.isfinite(sub_std).all(): 
                x, y = np.meshgrid(theta, theta)
                pos = np.dstack((x, y))
                rv = st.multivariate_normal(sub_mean, sub_std)
                plt.subplot(n_params, n_params, plot_idx)
                plt.contourf(x, y, rv.pdf(pos))
                plt.plot([true_params[0][i]], [true_params[0][j]], 'o', color='C1')
            plot_idx += 1        

In [ ]:
# set up a dict for saving the results 
save_data = True
path_to_save_folder = 'data/'
if save_data and os.path.exists(path_to_save_folder): 
    nrounds=1
    result_dict = dict(true_params=true_params, stats_obs=stats_obs, nrouns=nrounds, ntrain=ntrain,
                       posterior=dict(mean=mean, std=std))
    filename = os.path.join(path_to_save_folder, 
                           '{}_basic_ntrain{}'.format(time.time(), ntrain).replace('.', '') + '.p')
    with open(filename, 'wb') as handle:
        pickle.dump(result_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
theta = np.linspace(0, 5, 1000)
post_pdf = st.norm.pdf(x=theta, loc=mean, scale=std)

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(theta, post_pdf, label='$\hat{p}( theta | x=x_{obs})$')
plt.axvline(x=true_params[0], label='true theta', linestyle='--', color='C1')
plt.legend()
plt.xlabel('$R_{ee}$');